#**Optimización por el método de algoritmo de evolución diferencial para dos dimensiones**

In [ ]:
import numpy as np

# Definimos la función de Rastrigin
def rastrigin_function(x):
    x1, x2 = x
    return 20 + (x1)**2 + (x2)**2 - 10 * (np.cos(2*np.pi*x1) + np.cos(2*np.pi*x2))

In [ ]:
def differential_evolution(func, bounds, pop_size=50, F=0.5, CR=0.7, max_iter=1000, tol=1e-6):
    """
    Método diferencial de optimización.

    Parámetros:
    - func: función objetivo a optimizar.
    - bounds: límites de búsqueda para cada parámetro de la función.
    - pop_size: tamaño de la población.
    - F: factor de escala.
    - CR: tasa de cruce.
    - max_iter: número máximo de iteraciones.
    - tol: tolerancia para la convergencia.

    Retorna:
    - best_solution: la mejor solución encontrada.
    - best_fitness: el valor de la función objetivo en la mejor solución.
    - all_solutions: un array que guarda todas las soluciones encontradas en cada iteración.
    """
    # Determina la cantidad de parámetros a optimizar
    n_params = len(bounds)

    # Genera una población inicial de soluciones dentro de los límites especificados
    pop = np.random.rand(pop_size, n_params)
    min_b, max_b = np.asarray(bounds).T
    diff = np.fabs(min_b - max_b)
    pop_denorm = min_b + pop * diff

    # Calcula la aptitud de cada solución en la población inicial
    fitness = np.asarray([func(ind) for ind in pop_denorm])

    # Encuentra la mejor solución inicial
    best_idx = np.argmin(fitness)
    best_solution = pop_denorm[best_idx]
    best_fitness = fitness[best_idx]

    # Inicializa el array para guardar todas las soluciones
    all_solutions = [best_solution.copy()]

    # Realiza el proceso de optimización
    for i in range(max_iter):
        for j in range(pop_size):
            # Selecciona tres individuos distintos aleatoriamente
            idxs = [idx for idx in range(pop_size) if idx != j]
            a, b, c = pop[np.random.choice(idxs, 3, replace=False)]

            # Genera un mutante basado en la diferencia entre dos individuos y escala el factor F
            mutant = np.clip(a + F * (b - c), 0, 1)

            # Realiza el cruce entre el individuo actual y el mutante
            cross_points = np.random.rand(n_params) < CR
            if not np.any(cross_points):
                cross_points[np.random.randint(0, n_params)] = True
            trial = np.where(cross_points, mutant, pop[j])

            # Evalúa la aptitud de la solución propuesta
            trial_denorm = min_b + trial * diff
            f = func(trial_denorm)

            # Actualiza la población si se ha encontrado una solución mejor
            if f < fitness[j]:
                fitness[j] = f
                pop[j] = trial
                if f < best_fitness:
                    best_solution = trial_denorm
                    best_fitness = f

        # Guarda la mejor solución de esta iteración
        all_solutions.append(best_solution.copy())

        # Verifica si la población ha convergido (varianza de la aptitud baja)
        if np.std(fitness) < tol:
            break

    # Devuelve la mejor solución encontrada, su valor de la función objetivo y todas las soluciones
    return best_solution, best_fitness, np.array(all_solutions)

# Define los límites de búsqueda para Rastrigin en 2D
bounds = [(-5.12, 5.12), (-5.12, 5.12)]

# Ejecuta la optimización
best_solution, best_fitness, all_solutions = differential_evolution(rastrigin_function, bounds)

# Imprime la mejor solución encontrada y su valor de la función objetivo
print("Mejor solución encontrada:", best_solution)
print("Valor de la función objetivo en la mejor solución:", best_fitness)


Mejor solución encontrada: [-4.60024268e-06  5.43641385e-06]
Valor de la función objetivo en la mejor solución: 1.0061818045414839e-08


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import imageio

# Función para graficar y guardar las soluciones como imágenes
def plot_and_save_solutions(all_solutions, bounds, output_file):
    x_vals = np.linspace(bounds[0][0], bounds[0][1], 100)
    y_vals = np.linspace(bounds[1][0], bounds[1][1], 100)
    X, Y = np.meshgrid(x_vals, y_vals)
    Z = rastrigin_function([X, Y])

    images = []
    for i, solution in enumerate(all_solutions):
        plt.figure()
        plt.contourf(X, Y, Z, levels=100, cmap='viridis')
        plt.plot(solution[0], solution[1], 'ro')
        plt.title('Iteración {}'.format(i+1))
        plt.xlabel('X')
        plt.ylabel('Y')

        # Guardar la imagen
        filename = 'solution_{}.png'.format(i)
        plt.savefig(filename)
        images.append(filename)
        plt.close()

    # Crear el archivo GIF
    with imageio.get_writer(output_file, mode='I') as writer:
        for filename in images:
            image = imageio.imread(filename)
            writer.append_data(image)

# Define los límites de búsqueda para Rastrigin en 2D
bounds = [(-5.12, 5.12), (-5.12, 5.12)]

# Ejecuta la optimización y guarda todas las soluciones
best_solution, best_fitness, all_solutions = differential_evolution(rastrigin_function, bounds)

# Graficar y guardar las soluciones en un archivo GIF
plot_and_save_solutions(all_solutions, bounds, 'solutions.gif')


<ipython-input-75-b786ad1f87b0>:30: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


In [ ]:
from IPython.display import Video
from moviepy.editor import VideoFileClip

gif_path = 'solutions.gif'

# Convertir GIF a video
clip = VideoFileClip(gif_path)
video_path = "output.mp4"
clip.write_videofile(video_path, fps=1000, logger=None)

# Mostrar el video
Video(video_path, embed=True)